In [94]:
import torch
import random
import numpy as np
import re

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

PATH="/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity_100"
FASTBPE="/home/vzhekova/fastBPE/fast" # path to the fastBPE tool
FAST_ALIGN="/home/vzhekova/fast_align/build/fast_align" # path to the fast_align tool
TERCOM = "/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity_male/Perturbation-basedQE"

In [95]:
# check if we can connect to the GPU with PyTorch
if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print('Current device:', torch.cuda.get_device_name(device))
else:
    print('Failed to find GPU. Will use CPU.')
    device = 'cpu'

Current device: GeForce GTX 1080 Ti


In [96]:
%cd $PATH

/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity_100


# Translation English-German

In [8]:
MODELS="/export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble"
NBEST = 100
BEAM = 100

In [12]:
# Generate translations
# Beam search
!fairseq-generate data-bin_original_en-de  \
    --task translation \
    --source-lang en \
    --target-lang de \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 8 \
    --memory-efficient-fp16 \
    --remove-bpe > original_en-de.decode_Beam_10.log

2023-09-19 12:00:16 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

In [13]:
!fairseq-generate data-bin_disambiguated_male_en-de  \
    --task translation \
    --source-lang en \
    --target-lang de \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 8 \
    --memory-efficient-fp16 \
    --remove-bpe > disambiguated_male_en-de.decode_Beam_10.log

2023-09-19 12:03:30 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

In [14]:
!fairseq-generate data-bin_disambiguated_female_en-de  \
    --task translation \
    --source-lang en \
    --target-lang de \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 8 \
    --memory-efficient-fp16 \
    --remove-bpe > disambiguated_female_en-de.decode_Beam_10.log

print('Finished translation.')

2023-09-19 12:06:47 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

In [15]:
# 'LC_ALL=C sort -V' sorts the results in natural order 
!grep ^H original_en-de.decode_Beam_10.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_original.txt
!grep ^H disambiguated_male_en-de.decode_Beam_10.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_disambiguated_male.txt
!grep ^H disambiguated_female_en-de.decode_Beam_10.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_disambiguated_female.txt

# Backtranslation German-English

In [16]:
# Dividing tokenized text into subword units

!$FASTBPE applybpe bpe.hyp_original.de hyp_original.txt bpecodes.de
!$FASTBPE applybpe bpe.hyp_disambiguated_male.de hyp_disambiguated_male.txt bpecodes.de
!$FASTBPE applybpe bpe.hyp_disambiguated_female.de hyp_disambiguated_female.txt bpecodes.de


print('Finished subword.')

Loading codes from bpecodes.de ...
Read 30000 codes from the codes file.
Loading vocabulary from hyp_original.txt ...
Read 249165 words (3939 unique) from text file.
Applying BPE to hyp_original.txt ...
Modified 249165 words from text file.
Loading codes from bpecodes.de ...
Read 30000 codes from the codes file.
Loading vocabulary from hyp_disambiguated_male.txt ...
Read 269736 words (3573 unique) from text file.
Applying BPE to hyp_disambiguated_male.txt ...
Modified 269736 words from text file.
Loading codes from bpecodes.de ...
Read 30000 codes from the codes file.
Loading vocabulary from hyp_disambiguated_female.txt ...
Read 255036 words (3895 unique) from text file.
Applying BPE to hyp_disambiguated_female.txt ...
Modified 255036 words from text file.
Finished subword.


In [17]:
!fairseq-preprocess \
    --source-lang de \
    --target-lang en \
    --only-source \
    --testpref bpe.hyp_original \
    --srcdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.de.txt \
    --tgtdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.en.txt \
    --destdir data-bin_original_de-en \
    --workers 8

!fairseq-preprocess \
    --source-lang de \
    --target-lang en \
    --only-source \
    --testpref bpe.hyp_disambiguated_male \
    --srcdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.de.txt \
    --tgtdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.en.txt \
    --destdir data-bin_disambiguated_male_de-en \
    --workers 8

!fairseq-preprocess \
    --source-lang de \
    --target-lang en \
    --only-source \
    --testpref bpe.hyp_disambiguated_female \
    --srcdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.de.txt \
    --tgtdict /export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble/dict.en.txt \
    --destdir data-bin_disambiguated_female_de-en \
    --workers 8

print('Finished preprocessing.')

2023-09-19 12:10:23 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin_original_de-en', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='de', srcd

In [21]:
MODELS="/export/data4/vzhekova/biases-data/De-En/wmt19.de-en.joined-dict.ensemble"
NBEST = 100
BEAM = 100

In [19]:
# Generate backtranslations
!fairseq-generate data-bin_original_de-en  \
    --task translation \
    --source-lang de \
    --target-lang en \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 8 \
    --memory-efficient-fp16 \
    --remove-bpe > original_de-en.decode_Beam_10_backtranslation.log

2023-09-19 12:11:06 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

In [22]:
!fairseq-generate data-bin_disambiguated_male_de-en  \
    --task translation \
    --source-lang de \
    --target-lang en \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 8 \
    --memory-efficient-fp16 \
    --remove-bpe > disambiguated_male_de-en.decode_Beam_10_backtranslation.log

2023-09-19 17:30:16 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

In [23]:
!fairseq-generate data-bin_disambiguated_female_de-en  \
    --task translation \
    --source-lang de \
    --target-lang en \
    --path $MODELS/model1.pt:$MODELS/model2.pt:$MODELS/model3.pt:$MODELS/model4.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 8 \
    --memory-efficient-fp16 \
    --remove-bpe > disambiguated_female_de-en.decode_Beam_10_backtranslation.log

print('Finished translation.')

2023-09-19 22:08:19 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [24]:
# 'LC_ALL=C sort -V' sorts the results in natural order 
!grep ^H original_de-en.decode_Beam_10_backtranslation.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_original_back.txt
!grep ^H disambiguated_male_de-en.decode_Beam_10_backtranslation.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_disambiguated_male_back.txt
!grep ^H disambiguated_female_de-en.decode_Beam_10_backtranslation.log | LC_ALL=C sort -V | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp_disambiguated_female_back.txt

In [25]:
# Detokenize text        
from sacremoses import MosesPunctNormalizer
from sacremoses import MosesTokenizer, MosesDetokenizer
from __future__ import print_function

md_en = MosesDetokenizer(lang='en')

with open('hyp_original_back.txt', encoding='utf8') as fin, open('original_back.txt','w', encoding='utf8') as fout:
    for line in fin:
        tokens = md_en.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)
        
with open('hyp_disambiguated_male_back.txt', encoding='utf8') as fin, open('disambiguated_male_back.txt','w', encoding='utf8') as fout:
    for line in fin:
        tokens = md_en.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)
        
with open('hyp_disambiguated_female_back.txt', encoding='utf8') as fin, open('disambiguated_female_back.txt','w', encoding='utf8') as fout:
    for line in fin:
        tokens = md_en.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)

print('Finished detokenizing.')

Finished detokenizing.


# Statistics on translations

In [26]:
# List with original source sentences
source = []
with open('en_original.txt', 'r') as fin:
    for line in fin:
        source.append(line.strip())
        
# List with disambiguated source sentences male
source_disambiguated_male = []
with open('en_disambiguated_male.txt', 'r') as fin:
    for line in fin:
        source_disambiguated_male.append(line.strip())
        
# List with disambiguated source sentences female
source_disambiguated_female = []
with open('en_disambiguated_female.txt', 'r') as fin:
    for line in fin:
        source_disambiguated_female.append(line.strip())        
    
# List with nbest sentences for every source in original
nbest_original = []
counter = 0
temp = []
with open('hyp_original.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 100):
            nbest_original.append(temp)
            counter = 0
            temp = []
            
# List with nbest sentences for every source in disambiguated male        
nbest_disambiguated_male = []
with open('hyp_disambiguated_male.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 100):
            nbest_disambiguated_male.append(temp)
            counter = 0
            temp = []
            
# List with nbest sentences for every source in disambiguated female        
nbest_disambiguated_female = []
with open('hyp_disambiguated_female.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 100):
            nbest_disambiguated_female.append(temp)
            counter = 0
            temp = []
            
print(len(source))
print(len(nbest_original))
print(len(nbest_disambiguated_male))
print(len(nbest_disambiguated_female))

335
335
335
335


## Count unique sentences

In [27]:
# Count unique sentences in source nbest list for each source sentence
def count_unique_sentences(nbest_sentences):
    unique_sent = []
    for source_nbest in nbest_sentences:
        num_values = len(set(source_nbest))
        #print(num_values)
        unique_sent.append(num_values)

    #print(unique_sent)
    return sum(unique_sent)/len(nbest_sentences) # average

In [28]:
# Value should be 10, because beam search generates 10 unique sentences
print(count_unique_sentences(nbest_original))

print(count_unique_sentences(nbest_disambiguated_male))

print(count_unique_sentences(nbest_disambiguated_female))

98.88358208955223
98.80597014925372
97.84179104477612


## Count unique words

In [29]:
# Count unique words in source nbest list for each source sentence of original
# !!! Method is slow
import spacy

def count_unique_words(nbest_sentences):
    sp = spacy.load('en_core_web_sm')
    stopwords = spacy.lang.en.stop_words.STOP_WORDS
    
    unique_words = []
    normalizer = 0 # should normalize based on total number of words, because disambiguated sentences have more words overall
    counter = 0
    for source_nbest in nbest_sentences:
        words = set()
        for sent in source_nbest:
            tokens = sp(sent)
            normalizer += len(tokens)
            for token in tokens:
                # if token.text not in stopwords:    # checking whether the word is a stop word
                    words.add(token.text)
        num_values = len(words)
        unique_words.append(num_values)

        counter += 1
        #print(counter)

    #print(unique_words)
    print('Normalizer: ' + str(normalizer/len(nbest_sentences)))
    return (sum(unique_words)/len(nbest_sentences), (sum(unique_words)/len(nbest_sentences))/(normalizer/len(nbest_sentences))) # (average, norm average)

In [30]:
print(count_unique_words(nbest_original))

print(count_unique_words(nbest_disambiguated_male))

print(count_unique_words(nbest_disambiguated_female))

Normalizer: 745.2805970149253
(53.53432835820895, 0.07183110438220204)
Normalizer: 806.4238805970149
(50.22686567164179, 0.06228345524001303)
Normalizer: 762.5104477611941
(54.026865671641794, 0.07085393495954055)


# Statistics on backtranslations

In [97]:
# List with original source sentences
source = []
with open('en_original.txt', 'r') as fin:
    for line in fin:
        source.append(line.strip())
        
# List with disambiguated source sentences male
source_disambiguated_male = []
with open('en_disambiguated_male.txt', 'r') as fin:
    for line in fin:
        source_disambiguated_male.append(line.strip())
        
# List with disambiguated source sentences female
source_disambiguated_female = []
with open('en_disambiguated_female.txt', 'r') as fin:
    for line in fin:
        source_disambiguated_female.append(line.strip())        
    
# List with nbest sentences for every source in original 
nbest_original = []
counter = 0
temp = []
with open('original_back.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 10000):
            nbest_original.append(temp)
            counter = 0
            temp = []
            
# List with nbest sentences for every source in disambiguated male
nbest_disambiguated_male = []
with open('disambiguated_male_back.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 10000):
            nbest_disambiguated_male.append(temp)
            counter = 0
            temp = []
            
# List with nbest sentences for every source in disambiguated female
nbest_disambiguated_female = []
with open('disambiguated_female_back.txt', 'r') as fin:
    for line in fin:
        temp.append(line.strip())
        counter += 1
        if (counter == 10000):
            nbest_disambiguated_female.append(temp)
            counter = 0
            temp = []
            
print(len(nbest_original))
print(len(nbest_disambiguated_male))
print(len(nbest_disambiguated_female))

335
335
335


## Source sentence reoccurrence

In [98]:
# Count how many of the source sentences reoccur in the backtranslation
def count_sentence_reoccurrence(source_sentences, nbest_sentences):
    results = []
    counter = 0
    for sent in source_sentences:
        matches = 0
        for target in nbest_sentences[counter]: 
            if (sent == target):
                matches += 1
        results.append(matches)  
        counter += 1
    
    print(sum(results)/len(source_sentences)) # average reoccurence per sentence for 100 backtranslations; x/100

    return sum(x > 0 for x in results)

In [99]:
print(count_sentence_reoccurrence(source, nbest_original))

print(count_sentence_reoccurrence(source_disambiguated_male, nbest_disambiguated_male))

print(count_sentence_reoccurrence(source_disambiguated_female, nbest_disambiguated_female))

72.51044776119403
329
48.58805970149254
330
21.84776119402985
281


## Ambiguous source words reoccurrence


In [103]:
# Extract ambiguous words from source sentences
ambiguous_words = [] 
with open('en_original.txt', 'r') as fin:
    for line in fin:
        tokens = line.split(" ")
        ambiguous_words.append(tokens[1].replace('\n', ''))
        
#print(ambiguous_words)
#print(len(ambiguous_words))

In [101]:
# Count how many of the ambiguous words reoccur in the backtranslation
def count_words_reoccurrence(ambiguous_words, nbest_sentences):
    results = []
    counter = 0
    for word in ambiguous_words:
        matches = 0
        for target in nbest_sentences[counter]: 
            if (word in target.split(" ")):
                matches += 1
        results.append(matches)  
        counter += 1
    
    print(sum(results)/len(ambiguous_words)) # average reoccurence per sentence for 100 backtranslations; x/100

    return sum(x > 0 for x in results)

In [102]:
print(count_words_reoccurrence(ambiguous_words, nbest_original))

print(count_words_reoccurrence(ambiguous_words, nbest_disambiguated_male))

print(count_words_reoccurrence(ambiguous_words, nbest_disambiguated_female))

4714.740298507462
335
4788.994029850746
335
4192.5402985074625
335


## Count unique sentences

In [37]:
print(count_unique_sentences(nbest_original))

print(count_unique_sentences(nbest_disambiguated_male))

print(count_unique_sentences(nbest_disambiguated_female))

3181.513432835821
3391.5522388059703
3424.976119402985


## Count unique words

In [38]:
# Takes very long!!!

print(count_unique_words(nbest_original))

print(count_unique_words(nbest_disambiguated_male))

print(count_unique_words(nbest_disambiguated_female))


KeyboardInterrupt



# Word alignement (source-translation)

- Input to fast_align must be tokenized and aligned into parallel sentences. 
- Line is a source language sentence and its target language translation, separated by a triple pipe symbol with leading and trailing white space (|||)

In [44]:
def build_alignment_input(sentencesN, sourceIn, targetIn, output):
    # List with original source sentences
    source = []
    with open(sourceIn, 'r') as fin:
        for line in fin:
            source.append(line.strip())

    # List with nbest sentences for every source in original 
    target = []
    counter = 0
    temp = []
    with open(targetIn, 'r') as fin:
        for line in fin:
            temp.append(line.strip())
            counter += 1
            if (counter == 100):
                target.append(temp)
                counter = 0
                temp = []

    #print(len(source))
    #print(len(target))           

    count = 0
    with open(output, 'w') as fout:
        while count < sentencesN:
            for hyp in target[count]:
                print(source[count] + ' ||| ' + hyp, end='\n', file=fout)
            count += 1

In [45]:
build_alignment_input(335, 'tok.en_original.en', 'hyp_original.txt', 'original_source-target_en-de.txt')
build_alignment_input(335, 'tok.en_disambiguated_male.en', 'hyp_disambiguated_male.txt', 'disambiguated_male_source-target_en-de.txt')
build_alignment_input(335, 'tok.en_disambiguated_female.en', 'hyp_disambiguated_female.txt', 'disambiguated_female_source-target_en-de.txt')

## fast_align

In [46]:
!$FAST_ALIGN -i original_source-target_en-de.txt -d -o -v > original_source-target_en-de_fast-aligned.txt
!$FAST_ALIGN -i disambiguated_male_source-target_en-de.txt -d -o -v > disambiguated_male_source-target_en-de_fast-aligned.txt
!$FAST_ALIGN -i disambiguated_female_source-target_en-de.txt -d -o -v > disambiguated_female_source-target_en-de_fast-aligned.txt

print("Finished alignment.")

ARG=i
ARG=d
ARG=o
ARG=v
INITIAL PASS 
.................................
expected target length = source length * 1.03694
ITERATION 1
.................................
  log_e likelihood: -5.16351e+06
  log_2 likelihood: -7.44937e+06
     cross entropy: 29.8974
        perplexity: 1e+09
      posterior p0: 0.08
 posterior al-feat: -0.196766
       size counts: 77
ITERATION 2
.................................
  log_e likelihood: -797565
  log_2 likelihood: -1.15064e+06
     cross entropy: 4.618
        perplexity: 24.5559
      posterior p0: 0.0269036
 posterior al-feat: -0.161889
       size counts: 77
  1  model al-feat: -0.160026 (tension=4)
  2  model al-feat: -0.16098 (tension=3.96273)
  3  model al-feat: -0.161449 (tension=3.94455)
  4  model al-feat: -0.161676 (tension=3.93574)
  5  model al-feat: -0.161786 (tension=3.93148)
  6  model al-feat: -0.16184 (tension=3.92943)
  7  model al-feat: -0.161865 (tension=3.92844)
  8  model al-feat: -0.161878 (tension=3.92796)
     final tens

In [90]:
import re

# Count unique translated words to the ambiguous words in translations per source sentence
def count_unique_words_alignment_translations(position, sentencesN, sourceIn, translationsIn, alignmentsIn, output):
    
    # Get positions of ambigous words
    positions_ambiguous_words = []
    for i in range(sentencesN):
        positions_ambiguous_words.append(position)
        
    # List with translations
    translations = []
    with open(translationsIn, 'r') as fin:
        for line in fin:
            translations.append(line.strip())
            
            
    
    # Extract alginments of ambiguous words
    lineNumber = 0
    counter = 0
    indices = [] # a list of lists of indices of translated words for each ambiguous word
    with open(alignmentsIn, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 100):
                lineNumber = 0
                counter += 1
            position = positions_ambiguous_words[counter] # exact position of ambiguous word
            regex = r"" + str(position) + r"-(\d)"
            if re.findall(regex, line): 
                indices.append([int(index) for index in re.findall(regex, line)])
            else:
                indices.append([999])
            lineNumber += 1

    #print(len(indices))
    #print(indices)

    lineNumber = 0
    translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
    translated_ambiguous_words = set() # set forces uniqueness
    for translation in translations:
        tokens = translation.split(' ')
        if 999 not in indices[lineNumber]:
            for ind in indices[lineNumber]:
                #print(lineNumber)
                #print(tokens[ind])
                #print(ind)
                translated_ambiguous_words.add(tokens[ind])
        lineNumber += 1
        if (lineNumber % 100 == 0):
                translations_ambiguous_words.append(translated_ambiguous_words)
                translated_ambiguous_words = set()

    #print(translations_ambiguous_words)
    #print(len(translations_ambiguous_words))
    
    # Add results to file
    ambiguous_words = []
    source = []
    with open(sourceIn, 'r') as fin:
        for line in fin:
            source.append(line.strip())
            tokens = line.split(' ')
            ambiguous_words.append(tokens[position])

    count = 0                
    with open(output, 'w') as fout:
        while count < sentencesN:
            print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(translations_ambiguous_words[count]), end='\n', file=fout)
            count += 1

    unique_translations = 0
    for set_words in translations_ambiguous_words:
        
        ############################################################
        # remove gender info; removing "in" and "e" endings in words
        set_words_new = set()
        for word in set_words:
            word_new = re.sub("in$|e$", "", word)
            #print(word_new)
            set_words_new.add(word_new)
        #print(set_words_new)
        ############################################################
        
        unique_translations += len(set_words)
        
    #print(unique_translations)
    return unique_translations/sentencesN # average

In [91]:
print(count_unique_words_alignment_translations(1, 335, 'tok.en_original.en', 'hyp_original.txt', 'original_source-target_en-de_fast-aligned.txt', 'unique-words_translations_original.txt'))
print('======')
print(count_unique_words_alignment_translations(2, 335, 'tok.en_disambiguated_male.en', 'hyp_disambiguated_male.txt', 'disambiguated_male_source-target_en-de_fast-aligned.txt', 'unique-words_translations_disambiguated_male.txt')) # positions is 2 because of gender word
print('======')
print(count_unique_words_alignment_translations(2, 335, 'tok.en_disambiguated_female.en', 'hyp_disambiguated_female.txt', 'disambiguated_female_source-target_en-de_fast-aligned.txt', 'unique-words_translations_disambiguated_female.txt'))

12.373134328358208
13.695522388059702
14.75820895522388


In [24]:
import re

# Count unique translated words to the ambiguous words in translations per source sentence
def count_unique_words_alignment_translations_TEST(position, sentencesN, sourceIn, translationsIn, alignmentsIn, output):
    
    # Get positions of ambigous words
    positions_ambiguous_words = []
    for i in range(sentencesN):
        positions_ambiguous_words.append(position)
        
    # List with translations
    translations = []
    with open(translationsIn, 'r') as fin:
        for line in fin:
            translations.append(line.strip())
            
            
    
    # Extract alginments of ambiguous words
    lineNumber = 0
    counter = 0
    indices = [] # a list of lists of indices of translated words for each ambiguous word
    with open(alignmentsIn, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 10):
                lineNumber = 0
                counter += 1
            position = positions_ambiguous_words[counter] # exact position of ambiguous word
            regex = r"" + str(position) + r"-(\d)"
            if re.findall(regex, line): 
                indices.append([int(index) for index in re.findall(regex, line)])
            else:
                indices.append([999])
            lineNumber += 1

    #print(len(indices))
    #print(indices)

    lineNumber = 0
    translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
    translated_ambiguous_words = set() # set forces uniqueness
    for translation in translations:
        tokens = translation.split(' ')
        if 999 not in indices[lineNumber]:
            translated_ambiguous_words.add(' '.join([tokens[ind] for ind in indices[lineNumber]])) # join multiple aligned words
        lineNumber += 1
        if (lineNumber % 10 == 0):
                translations_ambiguous_words.append(translated_ambiguous_words)
                translated_ambiguous_words = set()

    #print(translations_ambiguous_words)
    #print(len(translations_ambiguous_words))
    
    # Add results to file
    ambiguous_words = []
    source = []
    with open(sourceIn, 'r') as fin:
        for line in fin:
            source.append(line.strip())
            tokens = line.split(' ')
            ambiguous_words.append(tokens[position])

    count = 0                
    with open(output, 'w') as fout:
        while count < sentencesN:
            print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(translations_ambiguous_words[count]), end='\n', file=fout)
            count += 1

    unique_translations = 0
    for set_words in translations_ambiguous_words:
        
        ############################################################
        # remove gender info; removing "in" and "e" endings in words
        set_words_new = set()
        for word in set_words:
            word_new = re.sub("in$|e$", "", word)
            #print(word_new)
            set_words_new.add(word_new)
        #print(set_words_new)
        ############################################################
        
        unique_translations += len(set_words_new)
        
    #print(unique_translations)
    return unique_translations/sentencesN # average

In [25]:
print(count_unique_words_alignment_translations_TEST(1, 335, 'tok.en_original.en', 'hyp_original.txt', 'original_source-target_en-de_fast-aligned.txt', 'unique-words_translations_original_TEST.txt'))
print('======')
print(count_unique_words_alignment_translations_TEST(2, 335, 'tok.en_disambiguated_male.en', 'hyp_disambiguated_male.txt', 'disambiguated_male_source-target_en-de_fast-aligned.txt', 'unique-words_translations_disambiguated_male_TEST.txt')) # positions is 2 because of gender word
print('======')
print(count_unique_words_alignment_translations_TEST(2, 335, 'tok.en_disambiguated_female.en', 'hyp_disambiguated_female.txt', 'disambiguated_female_source-target_en-de_fast-aligned.txt', 'unique-words_translations_disambiguated_female_TEST.txt'))

2.6
2.6029850746268655
2.773134328358209


## awesome_align

In [49]:
# ??? How to set model correctly
# MODELS="/export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble"
!awesome-align \
    --output_file "original_source-target_en-de_awesome-aligned.txt" \
    --data_file "original_source-target_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

!awesome-align \
    --output_file "disambiguated_male_source-target_en-de_awesome-aligned.txt" \
    --data_file "disambiguated_male_source-target_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

!awesome-align \
    --output_file "disambiguated_female_source-target_en-de_awesome-aligned.txt" \
    --data_file "disambiguated_female_source-target_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

print("Finished alignment.")

Loading the dataset...
Extracting: 33500it [00:35, 936.57it/s] 
Loading the dataset...
Extracting: 33500it [01:03, 529.67it/s]
Loading the dataset...
Extracting: 33500it [01:05, 509.64it/s]
Finished alignment.


In [93]:
print(count_unique_words_alignment_translations(1, 335, 'tok.en_original.en', 'hyp_original.txt', 'original_source-target_en-de_awesome-aligned.txt', 'unique-words_translations_original.txt'))
print('======')
print(count_unique_words_alignment_translations(2, 335, 'tok.en_disambiguated_male.en', 'hyp_disambiguated_male.txt', 'disambiguated_male_source-target_en-de_awesome-aligned.txt', 'unique-words_translations_disambiguated_male.txt')) # positions is 2 because of gender word
print('======')
print(count_unique_words_alignment_translations(2, 335, 'tok.en_disambiguated_female.en', 'hyp_disambiguated_female.txt', 'disambiguated_female_source-target_en-de_awesome-aligned.txt', 'unique-words_translations_disambiguated_female.txt'))

10.808955223880597
12.116417910447762
13.119402985074627


In [92]:
print(count_unique_words_alignment_translations_TEST(1, 335, 'tok.en_original.en', 'hyp_original.txt', 'original_source-target_en-de_awesome-aligned.txt', 'unique-words_translations_original_TEST.txt'))
print('======')
print(count_unique_words_alignment_translations_TEST(2, 335, 'tok.en_disambiguated_male.en', 'hyp_disambiguated_male.txt', 'disambiguated_male_source-target_en-de_awesome-aligned.txt', 'unique-words_translations_disambiguated_male_TEST.txt')) # positions is 2 because of gender word
print('======')
print(count_unique_words_alignment_translations_TEST(2, 335, 'tok.en_disambiguated_female.en', 'hyp_disambiguated_female.txt', 'disambiguated_female_source-target_en-de_awesome-aligned.txt', 'unique-words_translations_disambiguated_female_TEST.txt'))

NameError: name 'count_unique_words_alignment_translations_TEST' is not defined

# Word alignement (translation-backtranslation)

## fast_align

- Input to fast_align must be tokenized and aligned into parallel sentences. 
- Line is a source language sentence and its target language translation, separated by a triple pipe symbol with leading and trailing white space (|||)

In [51]:
build_alignment_input(33500, 'hyp_original.txt', 'hyp_original_back.txt', 'original_translation-back_en-de.txt')
build_alignment_input(33500, 'hyp_disambiguated_male.txt', 'hyp_disambiguated_male_back.txt', 'disambiguated_male_translation-back_en-de.txt')
build_alignment_input(33500, 'hyp_disambiguated_female.txt', 'hyp_disambiguated_female_back.txt', 'disambiguated_female_translation-back_en-de.txt')

- Word alignement

In [52]:
!$FAST_ALIGN -i original_translation-back_en-de.txt -d -o -v > original_translation-back_en-de_fast-aligned.txt
!$FAST_ALIGN -i disambiguated_male_translation-back_en-de.txt -d -o -v > disambiguated_male_translation-back_en-de_fast-aligned.txt
!$FAST_ALIGN -i disambiguated_female_translation-back_en-de.txt -d -o -v > disambiguated_female_translation-back_en-de_fast-aligned.txt

print("Finished alignment.")

ARG=i
ARG=d
ARG=o
ARG=v
INITIAL PASS 
.................................................. [50000]
.................................................. [100000]
.................................................. [150000]
.................................................. [200000]
.................................................. [250000]
.................................................. [300000]
.................................................. [350000]
.................................................. [400000]
.................................................. [450000]
.................................................. [500000]
.................................................. [550000]
.................................................. [600000]
.................................................. [650000]
.................................................. [700000]
.................................................. [750000]
.................................................. [800000]
...

- Extract target backtranslated words

In [63]:
import re

# Count unique translated words to the ambiguous words in backtranslations per source sentence
def count_unique_words_alignment_backtranslations(position_word, sentencesN, sourceIn, backtranslationsIn, alignmentsIn_translation, alignmentsIn_backtranslation, output):

    # Extract the position of the translated ambiguous word from each sentence
    positions_ambiguous_words = []
    for i in range(sentencesN):
        positions_ambiguous_words.append(position_word)
       
    lineNumber = 0
    counter = 0
    positions_ambiguous_words_translations = [] # a list of lists of indices of translated words for each ambiguous word
    with open(alignmentsIn_translation, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 100):
                lineNumber = 0
                counter += 1
            position = positions_ambiguous_words[counter] # exact position of ambiguous word
            regex = r"" + str(position) + r"-(\d)"
            if re.findall(regex, line): 
                positions_ambiguous_words_translations.append([int(index) for index in re.findall(regex, line)])
            else:
                positions_ambiguous_words_translations.append([999])
            lineNumber += 1

    # List with backtranslations
    backtranslations = []
    with open(backtranslationsIn, 'r') as fin:
        for line in fin:
            backtranslations.append(line.strip())

    lineNumber = 0
    counter = 0
    indices = []
    with open(alignmentsIn_backtranslation, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 100):
                lineNumber = 0
                counter += 1
            positions = positions_ambiguous_words_translations[counter] # exact positions of ambiguous words
            list_indices = []
            for position in positions:
                regex = r"" + str(position) + r"-(\d)"
                if re.findall(regex, line): 
                    list_indices.extend([int(index) for index in re.findall(regex, line)])
                else:
                    list_indices.extend([999])
            indices.append(list_indices)
            lineNumber += 1
    
    #print(len(indices))
    #print(indices)

    lineNumber = 0
    backtranslations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
    backtranslated_ambiguous_words = set() # set forces uniqueness
    for backtranslation in backtranslations:
        tokens = backtranslation.split(' ')
        if 999 not in indices[lineNumber]:
            for ind in indices[lineNumber]:
                #print(lineNumber)
                #print(tokens[ind])
                #print(ind)
                backtranslated_ambiguous_words.add(tokens[ind])
        lineNumber += 1
        if (lineNumber % 100 == 0):
                backtranslations_ambiguous_words.append(backtranslated_ambiguous_words)
                backtranslated_ambiguous_words = set()


    #print(backtranslations_ambiguous_words)
    print(len(backtranslations_ambiguous_words))

    # Here we need to merge the sets for every 10 sets, because we want to see unique words in the nbest 100 backtranslation
    backtranslations_ambiguous_words_reduced = []
    backtranslated_ambiguous_words = set() # set forces uniqueness
    counter = 0
    for set_words in backtranslations_ambiguous_words:
        backtranslated_ambiguous_words.update(set_words)
        counter += 1
        if (counter % 100 == 0):
            backtranslations_ambiguous_words_reduced.append(backtranslated_ambiguous_words)
            backtranslated_ambiguous_words = set()

    print(len(backtranslations_ambiguous_words_reduced)) 
    
    # Add results to file

    ambiguous_words = []
    source = []
    with open(sourceIn, 'r') as fin:
        for line in fin:
            source.append(line.strip())
            tokens = line.split(' ')
            ambiguous_words.append(tokens[position_word])

    count = 0                
    with open(output, 'w') as fout:
        while count < sentencesN:
            print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(backtranslations_ambiguous_words_reduced[count]), end='\n', file=fout)
            count += 1

    unique_backtranslations = 0
    for set_words in backtranslations_ambiguous_words_reduced:
        unique_backtranslations += len(set_words)
        
    return unique_backtranslations/sentencesN

In [64]:
print(count_unique_words_alignment_backtranslations(1, 335, 'tok.en_original.en', 'hyp_original_back.txt', 'original_source-target_en-de_fast-aligned.txt', 'original_translation-back_en-de_fast-aligned.txt', 'unique-words_backtranslations_original.txt'))
print(count_unique_words_alignment_backtranslations(2, 335, 'tok.en_disambiguated_male.en', 'hyp_disambiguated_male_back.txt', 'disambiguated_male_source-target_en-de_fast-aligned.txt', 'disambiguated_male_translation-back_en-de_fast-aligned.txt', 'unique-words_backtranslations_disambiguated_male.txt')) # positions is 2 because of gender word
print(count_unique_words_alignment_backtranslations(2, 335, 'tok.en_disambiguated_female.en', 'hyp_disambiguated_female_back.txt', 'disambiguated_female_source-target_en-de_fast-aligned.txt', 'disambiguated_female_translation-back_en-de_fast-aligned.txt', 'unique-words_backtranslations_disambiguated_female.txt'))

33500
335
147.9313432835821
33500
335
148.33731343283583
33500
335
163.5283582089552


In [36]:
import re

# Count unique translated words to the ambiguous words in backtranslations per source sentence
def count_unique_words_alignment_backtranslations_TEST(position, sentencesN, sourceIn, backtranslationsIn, alignmentsIn_translation, alignmentsIn_backtranslation, output):
    
    # Extract the position of the translated ambiguous word from each sentence
    positions_ambiguous_words = []
    for i in range(sentencesN):
        positions_ambiguous_words.append(position)
       
    lineNumber = 0
    counter = 0
    positions_ambiguous_words_translations = [] # a list of lists of indices of translated words for each ambiguous word
    with open(alignmentsIn_translation, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 10):
                lineNumber = 0
                counter += 1
            position = positions_ambiguous_words[counter] # exact position of ambiguous word
            regex = r"" + str(position) + r"-(\d)"
            if re.findall(regex, line): 
                positions_ambiguous_words_translations.append([int(index) for index in re.findall(regex, line)])
            else:
                positions_ambiguous_words_translations.append([999])
            lineNumber += 1
    
    # List with backtranslations
    backtranslations = []
    with open(backtranslationsIn, 'r') as fin:
        for line in fin:
            backtranslations.append(line.strip())

    lineNumber = 0
    counter = 0
    indices = []
    with open(alignmentsIn_backtranslation, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 10):
                lineNumber = 0
                counter += 1
            positions = positions_ambiguous_words_translations[counter] # exact positions of ambiguous words
            list_indices = []
            for position in positions:
                regex = r"" + str(position) + r"-(\d)"
                if re.findall(regex, line): 
                    list_indices.extend([int(index) for index in re.findall(regex, line)])
                else:
                    list_indices.extend([999])
            indices.append(list_indices)
            lineNumber += 1

    #print(len(indices))
    #print(indices)

    lineNumber = 0
    backtranslations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
    backtranslated_ambiguous_words = set() # set forces uniqueness
    for backtranslation in backtranslations:
        tokens = backtranslation.split(' ')
        if 999 not in indices[lineNumber]:
            backtranslated_ambiguous_words.add(' '.join([tokens[ind] for ind in indices[lineNumber]])) # join multiple aligned words
        lineNumber += 1
        if (lineNumber % 10 == 0):
                backtranslations_ambiguous_words.append(backtranslated_ambiguous_words)
                backtranslated_ambiguous_words = set()



    #print(backtranslations_ambiguous_words)
    print(len(backtranslations_ambiguous_words))

    # Here we need to merge the sets for every 10 sets, because we want to see unique words in the nbest 100 backtranslation
    backtranslations_ambiguous_words_reduced = []
    backtranslated_ambiguous_words = set() # set forces uniqueness
    counter = 0
    for set_words in backtranslations_ambiguous_words:
        backtranslated_ambiguous_words.update(set_words)
        counter += 1
        if (counter % 10 == 0):
            backtranslations_ambiguous_words_reduced.append(backtranslated_ambiguous_words)
            backtranslated_ambiguous_words = set()

    print(len(backtranslations_ambiguous_words_reduced)) 
    
    # Add results to file

    ambiguous_words = []
    source = []
    with open(sourceIn, 'r') as fin:
        for line in fin:
            source.append(line.strip())
            tokens = line.split(' ')
            ambiguous_words.append(tokens[position])

    count = 0                
    with open(output, 'w') as fout:
        while count < sentencesN:
            print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(backtranslations_ambiguous_words_reduced[count]), end='\n', file=fout)
            count += 1

    unique_backtranslations = 0
    for set_words in backtranslations_ambiguous_words_reduced:
        unique_backtranslations += len(set_words)
        
    return unique_backtranslations/sentencesN

In [37]:
print(count_unique_words_alignment_backtranslations_TEST(1, 335, 'tok.en_original.en', 'hyp_original_back.txt', 'original_source-target_en-de_fast-aligned.txt', 'original_translation-back_en-de_fast-aligned.txt', 'unique-words_backtranslations_original_TEST.txt'))
print(count_unique_words_alignment_backtranslations_TEST(2, 335, 'tok.en_disambiguated_male.en', 'hyp_disambiguated_male_back.txt', 'disambiguated_male_source-target_en-de_fast-aligned.txt', 'disambiguated_male_translation-back_en-de_fast-aligned.txt', 'unique-words_backtranslations_disambiguated_male_TEST.txt')) # positions is 2 because of gender word
print(count_unique_words_alignment_backtranslations_TEST(2, 335, 'tok.en_disambiguated_female.en', 'hyp_disambiguated_female_back.txt', 'disambiguated_female_source-target_en-de_fast-aligned.txt', 'disambiguated_female_translation-back_en-de_fast-aligned.txt', 'unique-words_backtranslations_disambiguated_female_TEST.txt'))

3350
335
9.829850746268656
3350
335
9.322388059701492
3350
335
11.623880597014926


## awesome_align

- Extract the position of the translated ambiguous word from each sentence

- Word alignement

In [65]:
# ??? How to set model correctly
# MODELS="/export/data4/vzhekova/biases-data/En-De/wmt19.en-de.joined-dict.ensemble"
!awesome-align \
    --output_file "original_translation-back_en-de_awesome-aligned.txt" \
    --data_file "original_translation-back_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

!awesome-align \
    --output_file "disambiguated_male_translation-back_en-de_awesome-aligned.txt" \
    --data_file "disambiguated_male_translation-back_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

!awesome-align \
    --output_file "disambiguated_female_translation-back_en-de_awesome-aligned.txt" \
    --data_file "disambiguated_female_translation-back_en-de.txt" \
    --model_name_or_path bert-base-multilingual-cased \
    --extraction 'softmax' \
    --batch_size 32

print("Finished alignment.")

Loading the dataset...
Extracting: 3350000it [1:44:05, 536.39it/s] 
Loading the dataset...
Extracting: 161248it [05:18, 391.31it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Extracting: 2421984it [1:16:49, 396.80it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Extracting: 3350000it [1:44:13, 535.73it/s] 
Loading the dataset...
Extracting: 1538016it [46:43, 620.63it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Extracting: 2298560it [1:09:38, 777.14it/s] 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Extracting: 3350000it [1:41:22, 550.76it/s] 
Finished alignment.


- Extract target backtranslated words

In [66]:
print(count_unique_words_alignment_backtranslations(1, 335, 'tok.en_original.en', 'hyp_original_back.txt', 'original_source-target_en-de_awesome-aligned.txt', 'original_translation-back_en-de_awesome-aligned.txt', 'unique-words_backtranslations_original.txt'))
print(count_unique_words_alignment_backtranslations(2, 335, 'tok.en_disambiguated_male.en', 'hyp_disambiguated_male_back.txt', 'disambiguated_male_source-target_en-de_awesome-aligned.txt', 'disambiguated_male_translation-back_en-de_awesome-aligned.txt', 'unique-words_backtranslations_disambiguated_male.txt')) # positions is 2 because of gender word
print(count_unique_words_alignment_backtranslations(2, 335, 'tok.en_disambiguated_female.en', 'hyp_disambiguated_female_back.txt', 'disambiguated_female_source-target_en-de_awesome-aligned.txt', 'disambiguated_female_translation-back_en-de_awesome-aligned.txt', 'unique-words_backtranslations_disambiguated_female.txt'))

33500
335
120.08358208955224
33500
335
121.96119402985074
33500
335
136.1373134328358


In [39]:
print(count_unique_words_alignment_backtranslations_TEST(1, 335, 'tok.en_original.en', 'hyp_original_back.txt', 'original_source-target_en-de_awesome-aligned.txt', 'original_translation-back_en-de_awesome-aligned.txt', 'unique-words_backtranslations_original_TEST.txt'))
print(count_unique_words_alignment_backtranslations_TEST(2, 335, 'tok.en_disambiguated_male.en', 'hyp_disambiguated_male_back.txt', 'disambiguated_male_source-target_en-de_awesome-aligned.txt', 'disambiguated_male_translation-back_en-de_awesome-aligned.txt', 'unique-words_backtranslations_disambiguated_male_TEST.txt')) # positions is 2 because of gender word
print(count_unique_words_alignment_backtranslations_TEST(2, 335, 'tok.en_disambiguated_female.en', 'hyp_disambiguated_female_back.txt', 'disambiguated_female_source-target_en-de_awesome-aligned.txt', 'disambiguated_female_translation-back_en-de_awesome-aligned.txt', 'unique-words_backtranslations_disambiguated_female_TEST.txt'))

3350
335
8.071641791044776
3350
335
7.916417910447761
3350
335
8.868656716417911


# Word alignement (translation-translation)

## Tercom alignement (borrowed from Tu)
- https://github.com/TuAnh23/Perturbation-basedQE/blob/master/align_and_analyse_ambiguous_trans.py#L54-L92

In [ ]:
!git clone https://github.com/TuAnh23/Perturbation-basedQE.git

In [114]:
%cd $TERCOM

/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity_male/Perturbation-basedQE


In [115]:
import align_and_analyse_ambiguous_trans as tercom
import pandas as pd

def count_unique_words_tercom_alignment(position, sentencesN, sourceIn, backtranslationsIn):
    # List with source sentences; output 100 times to match backtranslation size
    source = []
    with open(PATH + "/" + sourceIn, 'r') as fin:
        for line in fin:
            for i in range(100): # append the source sentence 100 times to match backtranslations later
                source.append(line.strip().split()) # split() tokenizes the sentence, because tercom expects tokens     

    print(len(source))

    # List with original backtranslations
    backtranslations = []
    with open(PATH + "/" + backtranslationsIn, 'r') as fin:
        for line in fin:
            backtranslations.append(line.strip().split())

    print(len(backtranslations))
    
    # Generate alignments
    alignments = tercom.tercom_alignment(source, backtranslations)
    
    # Extract the position of the translated ambiguous word from each sentence
    positions_ambiguous_words = []
    for i in range(sentencesN):
        positions_ambiguous_words.append(position)
        
    # Extract target translated words to source words
    lineNumber = 0
    counter = 0
    indices = []
    for align in alignments:
        position = positions_ambiguous_words[counter] # exact position of ambiguous word
        indices.append([item[1] for item in (item for item in align if not(pd.isna(item[0]))) if item[0] == position][0])
        lineNumber += 1
        if (lineNumber % 100 == 0):
            counter += 1

    print(len(indices))

    lineNumber = 0
    translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
    translated_ambiguous_words = set() # set forces uniqueness
    for backtranslation in backtranslations:
        backtranslation_index = backtranslations.index(backtranslation)
        if not(pd.isna(indices[backtranslation_index])):
            translated_ambiguous_words.add(backtranslation[indices[backtranslation_index]])
        lineNumber += 1
        if (lineNumber % 100 == 0):
            translations_ambiguous_words.append(translated_ambiguous_words)
            translated_ambiguous_words = set()

    #print(translations_ambiguous_words)
    #print(len(translations_ambiguous_words))

    unique_translations = 0
    for set_words in translations_ambiguous_words:
        unique_translations += len(set_words)
        
    return unique_translations/sentencesN

In [116]:
print(count_unique_words_tercom_alignment(1, 335, 'tok.en_original.en', 'hyp_original_back.txt'))
print(count_unique_words_tercom_alignment(2, 335, 'tok.en_disambiguated_male.en', 'hyp_disambiguated_male_back.txt'))
print(count_unique_words_tercom_alignment(2, 335, 'tok.en_disambiguated_female.en', 'hyp_disambiguated_female_back.txt'))

33500
33500
33500
8.017910447761194
33500
33500
33500
7.898507462686567
33500
33500
33500
9.817910447761195


In [117]:
%cd $PATH

/export/data4/vzhekova/biases-data/Test_De/Statistics/Full_ambiguity


# Word occurrence

## Translation

In [71]:
def extract_word_translations(filename_tokenized, filename_translations, filename_out, filename_alignments):
    """
    Match alignment indices from translation to backtranslation
    """
    
    # Extract alignement indices from translation
    indices_translation = []
    with open(filename_alignments, 'r') as alignments:
        for line in alignments:
            alignement_tokens = line.split()
            indices_line = []
            for i in range(0, len(alignement_tokens)):    
                regex = r"" + str(i) + r"-(\d)"
                if re.findall(regex, line): 
                    indices_line.append([int(index) for index in re.findall(regex, line)])
                else:
                    indices_line.append([999])
            indices_translation.append(indices_line)
    
    # List with lengths of the source sentences
    source_lengths = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source_lengths.append(len(line.strip().split()))

    #print(source_lengths)

    # List with backtranslations
    translations = []
    with open(filename_translations, 'r') as fin:
         for line in fin:
                translations.append(line.split())

    #print(backtranslations)

    target_words = [] # list containing lists with translation sets for every word in the source sentences; length 335
    counter = 0
    for i in range(0, 335): # for every source sentence
        source_sent = []
        for j in range(0, source_lengths[i]): # for every word in the source sentence
            words_set = set()
            for  f in range(0, 99):
                alignments = indices_translation[counter + f]        
                if (j < len(alignments)):
                    for index in alignments[j]:
                        if index != 999:
                            if (index < len(translations[counter + f])):
                                 words_set.add(translations[counter + f][index])
            source_sent.append(words_set)
        target_words.append(source_sent)
        counter += 100

    #print(len(target_words))

    # Add results to file

    # List with source sentences
    source = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source.append(line.strip())

    count = 0                
    with open(filename_out, 'w') as fout:
        while count < 335:
            print(source[count] + ' | ' + str(target_words[count]), end='\n', file=fout)
            count += 1

In [72]:
def count_word_translations(filename_tokenized, filename_translations, filename_out, filename_alignments):
    """
    Match alignement indices from translation to backtranslation
    """
    
    # Extract alignement indices from translation
    indices_translation = []
    with open(filename_alignments, 'r') as alignments:
        for line in alignments:
            alignement_tokens = line.split()
            indices_line = []
            for i in range(0, len(alignement_tokens)):    
                regex = r"" + str(i) + r"-(\d)"
                if re.findall(regex, line): 
                    indices_line.append([int(index) for index in re.findall(regex, line)])
                else:
                    indices_line.append([999])
            indices_translation.append(indices_line)
    
    # List with lengths of the source sentences
    source_lengths = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source_lengths.append(len(line.strip().split()))

    #print(source_lengths)

    # List with backtranslations
    translations = []
    with open(filename_translations, 'r') as fin:
         for line in fin:
                translations.append(line.split())

    #print(backtranslations)

    target_words = [] # list containing lists with translation sets for every word in the source sentences; length 335
    counter = 0
    for i in range(0, 335): # for every source sentence
        source_sent = []
        for j in range(0, source_lengths[i]): # for every word in the source sentence
            words_set = set()
            for  f in range(0, 99):
                alignments = indices_translation[counter + f]        
                if (j < len(alignments)):
                    for index in alignments[j]:
                        if index != 999:
                            if (index < len(translations[counter + f])):
                                 words_set.add(translations[counter + f][index])
            source_sent.append(words_set)
        target_words.append(source_sent)
        counter += 100

    #print(len(target_words))

    # Add results to file

    # List with source sentences
    source = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source.append(line.strip())

    count = 0
    occurrences = []
    with open(filename_out, 'w') as fout:
        while count < 335:
            occurrences.append([len(target_set) for target_set in target_words[count]])
            print(source[count] + ' | ' + str([len(target_set) for target_set in target_words[count]]), end='\n', file=fout)
            count += 1
    
    return occurrences

- Investigate the variability of the remaining sentence without the ambiguos word

In [73]:
def uniqueness_rest_of_sentence(position, sentencesN, occurrences):
    
    # Get positions of ambigous words
    positions_ambiguous_words = []
    for i in range(sentencesN):
        positions_ambiguous_words.append(position)
        
    # Sum over all sentences and build average     
    counter = 0
    sum_sent = 0
    for occur in occurrences:
        position = positions_ambiguous_words[counter]
        sum_sent += (sum(occur) - occur[position])/(len(occur) - 1) # sum for every sentence
        counter += 1
        
    return sum_sent/sentencesN

In [74]:
extract_word_translations('tok.en_original.en', 'hyp_original.txt', 'translations_words_original.txt', 'original_source-target_en-de_awesome-aligned.txt')
occurrence_original = count_word_translations('tok.en_original.en', 'hyp_original.txt', 'translations_words_original_occurrence.txt', 'original_source-target_en-de_awesome-aligned.txt')
print(uniqueness_rest_of_sentence(1, 335, occurrence_original))

extract_word_translations('tok.en_disambiguated_male.en', 'hyp_disambiguated_male.txt', 'translations_words_disambiguated_male.txt', 'disambiguated_male_source-target_en-de_awesome-aligned.txt')
occurrence_male = count_word_translations('tok.en_disambiguated_male.en', 'hyp_disambiguated_male.txt', 'translations_words_disambiguated_male_occurrence.txt', 'disambiguated_male_source-target_en-de_awesome-aligned.txt')
print(uniqueness_rest_of_sentence(2, 335, occurrence_male))

extract_word_translations('tok.en_disambiguated_female.en', 'hyp_disambiguated_female.txt', 'translations_words_disambiguated_female.txt', 'disambiguated_female_source-target_en-de_awesome-aligned.txt')
occurrence_female = count_word_translations('tok.en_disambiguated_female.en', 'hyp_disambiguated_female.txt', 'translations_words_disambiguated_female_occurrence.txt', 'disambiguated_female_source-target_en-de_awesome-aligned.txt')
print(uniqueness_rest_of_sentence(2, 335, occurrence_female))


6.52295119766586
5.391120040821535
5.851519102787761


## Backtranslation

In [75]:
def extract_alignment_indices_backtranslation(filename_translations, filename_backtranslations):
    """
    Extract alignment indices
    """
    
    # Extract alignement indices from translation
    indices_translation = []
    with open(filename_translations, 'r') as alignments:
        for line in alignments:
            alignement_tokens = line.split()
            indices_line = []
            for i in range(0, len(alignement_tokens)):    
                regex = r"" + str(i) + r"-(\d)"
                if re.findall(regex, line): 
                    indices_line.append([int(index) for index in re.findall(regex, line)])
                else:
                    indices_line.append([999])
            indices_translation.append(indices_line)
       
    # Match alignement indices from translation to backtranslation
    lineNumber = 0
    counter = 0
    indices_backtranslation = []
    with open(filename_backtranslations, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 10000):
                lineNumber = 0
                counter += 1
            alignement_tokens = line.split()
            indices_line = []
            for index_list in indices_translation[counter]:
                index_matches = []
                for index in index_list:
                    regex = r"" + str(index) + r"-(\d)"
                    if re.findall(regex, line): 
                        index_matches.extend([int(i) for i in re.findall(regex, line)])
                    else:
                        index_matches.extend([999])
                indices_line.append(index_matches)
            indices_backtranslation.append(indices_line)
            lineNumber += 1 
    return indices_backtranslation

In [76]:
def extract_word_backtranslations(filename_tokenized, filename_backtranslations, filename_out, indices_backtranslation):
    """
    Match alignement indices from translation to backtranslation
    """
    
    # List with lengths of the source sentences
    source_lengths = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source_lengths.append(len(line.strip().split()))

    #print(source_lengths)

    # List with backtranslations
    backtranslations = []
    with open(filename_backtranslations, 'r') as fin:
         for line in fin:
                backtranslations.append(line.split())

    #print(backtranslations)

    target_words = [] # list containing lists with translation sets for every word in the source sentences; length 335
    counter = 0
    for i in range(0, 335): # for every source sentence
        source_sent = []
        for j in range(0, source_lengths[i]): # for every word in the source sentence
            words_set = set()
            for  f in range(0, 9999):
                alignments = indices_backtranslation[counter + f]        
                if (j < len(alignments)):
                    for index in alignments[j]:
                        if index != 999:
                            if (index < len(backtranslations[counter + f])):
                                 words_set.add(backtranslations[counter + f][index])
            source_sent.append(words_set)
        target_words.append(source_sent)
        counter += 10000

    #print(len(target_words))

    # Add results to file

    # List with source sentences
    source = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source.append(line.strip())

    count = 0                
    with open(filename_out, 'w') as fout:
        while count < 335:
            print(source[count] + ' | ' + str(target_words[count]), end='\n', file=fout)
            count += 1

In [77]:
def count_word_backtranslations(filename_tokenized, filename_backtranslations, filename_out, indices_backtranslation):
    """
    Match alignement indices from translation to backtranslation
    """
    
    # List with lengths of the source sentences
    source_lengths = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source_lengths.append(len(line.strip().split()))

    #print(source_lengths)

    # List with backtranslations
    backtranslations = []
    with open(filename_backtranslations, 'r') as fin:
         for line in fin:
                backtranslations.append(line.split())

    #print(backtranslations)

    target_words = [] # list containing lists with translation sets for every word in the source sentences; length 335
    counter = 0
    for i in range(0, 335): # for every source sentence
        source_sent = []
        for j in range(0, source_lengths[i]): # for every word in the source sentence
            words_set = set()
            for  f in range(0, 9999):
                alignments = indices_backtranslation[counter + f]        
                if (j < len(alignments)):
                    for index in alignments[j]:
                        if index != 999:
                            if (index < len(backtranslations[counter + f])):
                                 words_set.add(backtranslations[counter + f][index])
            source_sent.append(words_set)
        target_words.append(source_sent)
        counter += 10000

    #print(len(target_words))

    # Add results to file

    # List with source sentences
    source = []
    with open(filename_tokenized, 'r') as fin:
        for line in fin:
            source.append(line.strip())
       
    count = 0
    occurrences = []
    with open(filename_out, 'w') as fout:
        while count < 335:
            occurrences.append([len(target_set) for target_set in target_words[count]])
            print(source[count] + ' | ' + str([len(target_set) for target_set in target_words[count]]), end='\n', file=fout)
            count += 1
    
    return occurrences

In [78]:
indices_original = extract_alignment_indices_backtranslation('original_source-target_en-de_awesome-aligned.txt', 'original_translation-back_en-de_awesome-aligned.txt')
extract_word_backtranslations('tok.en_original.en', 'hyp_original_back.txt', 'backtranslations_words_original.txt', indices_original)
occurrences_original = count_word_backtranslations('tok.en_original.en', 'hyp_original_back.txt', 'backtranslations_words_original_occurrence.txt', indices_original)
print(uniqueness_rest_of_sentence(1, 335, occurrences_original))

indices_disambiguated_male = extract_alignment_indices_backtranslation('disambiguated_male_source-target_en-de_awesome-aligned.txt', 'disambiguated_male_translation-back_en-de_awesome-aligned.txt')
extract_word_backtranslations('tok.en_disambiguated_male.en', 'hyp_disambiguated_male_back.txt', 'backtranslations_words_disambiguated_male.txt', indices_disambiguated_male)
occurrences_male = count_word_backtranslations('tok.en_disambiguated_male.en', 'hyp_disambiguated_male_back.txt', 'backtranslations_words_disambiguated_male_occurrence.txt', indices_disambiguated_male)
print(uniqueness_rest_of_sentence(2, 335, occurrences_male))

indices_disambiguated_female = extract_alignment_indices_backtranslation('disambiguated_female_source-target_en-de_awesome-aligned.txt', 'disambiguated_female_translation-back_en-de_awesome-aligned.txt')
extract_word_backtranslations('tok.en_disambiguated_female.en', 'hyp_disambiguated_female_back.txt', 'backtranslations_words_disambiguated_female.txt', indices_disambiguated_female)
occurrences_female = count_word_backtranslations('tok.en_disambiguated_female.en', 'hyp_disambiguated_female_back.txt', 'backtranslations_words_disambiguated_female_occurrence.txt', indices_disambiguated_female)
print(uniqueness_rest_of_sentence(2, 335, occurrences_female))


68.00491853819773
66.38525414055266
63.406496488586065


# Gender statistics

In [79]:
from enum import Enum

class GENDER(Enum):
    """
    Enumerate possible genders.
    Ignore option resolves to words that should be ignored in particular language
    """
    male = 0
    female = 1
    neutral = 2
    unknown = 3
    ignore = 4
    
# ??? These are not always correct; 'der' could be Dativ or Genitiv for female, 'die' could be plural
# !!! There isn't always an article
DE_DETERMINERS = {"der": GENDER.male, "ein": GENDER.male, "dem": GENDER.male, "den": GENDER.male, 
                  "einen": GENDER.male, "des": GENDER.male, "er": GENDER.male, "seiner": GENDER.male,
                  "ihn": GENDER.male, "seinen": GENDER.male, "ihm": GENDER.male, "ihren": GENDER.male,
                  "die": GENDER.female, "eine": GENDER.female, "einer": GENDER.female, "seinem": GENDER.male,
                  "ihrem": GENDER.male, "sein": GENDER.male,
                  "sie": GENDER.female, "seine": GENDER.female, "ihrer": GENDER.female, 
                  "ihr": GENDER.neutral, "ihre": GENDER.neutral, "das": GENDER.neutral,
                  "jemanden": GENDER.neutral}

def get_german_determiners(words):
    """
    Get a list of (gender)
    given a list of words.
    """
    determiners = []
    for (word_ind, word) in enumerate(words):
        word = word.lower()
        if word in DE_DETERMINERS:
            determiners.append((DE_DETERMINERS[word].name))
    return determiners

In [80]:
dets = get_german_determiners(["dem"])
print(dets)

['male']


- Calculate gender based on the articles of unique words: how many of the sentences produce both genders, female and male

In [81]:
import re

# Extract articles of target tranlsated words
def extract_articles(position, sentencesN, translationsIn, alignmentsIn, sourceIn, output):
    
    # Extract the position of the translated ambiguous word from each sentence
    positions_ambiguous_words = []
    for i in range(sentencesN):
        positions_ambiguous_words.append(position)
    
    # List with original translations
    translations_original = []
    with open(translationsIn, 'r') as fin:
        for line in fin:
            translations_original.append(line.strip())


    lineNumber = 0
    counter = 0
    indices = [] # a list of lists of indices of translated words for each ambiguous word
    with open(alignmentsIn, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 100):
                lineNumber = 0
                counter += 1
            position = positions_ambiguous_words[counter] # exact position of ambiguous word
            regex = r"" + str(position) + r"-(\d)"
            if re.findall(regex, line): 
                indices.append([int(index) for index in re.findall(regex, line)])
            else:
                indices.append([999])
            lineNumber += 1

    #print(len(indices))
    #print(indices)

    lineNumber = 0
    translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
    translated_ambiguous_words = set() # set forces uniqueness
    for translation in translations_original:
        tokens = translation.split(' ')
        if 999 not in indices[lineNumber]:
            for ind in indices[lineNumber]:
                translated_ambiguous_words.add(tokens[0]) # extract articles; currently assume index 0 for article position, TODO
        lineNumber += 1
        if (lineNumber % 100 == 0):
                translations_ambiguous_words.append(translated_ambiguous_words)
                translated_ambiguous_words = set()


    #print(len(translations_ambiguous_words))
    
    # Add results to file

    # List with original source sentences
    source = []
    ambiguous_words = []
    with open(sourceIn, 'r') as fin:
        for line in fin:
            source.append(line.strip())
            tokens = line.split(' ')
            ambiguous_words.append(tokens[position])

    count = 0  
    genders = []
    male = []
    female = []
    with open(output, 'w') as fout:
        while count < sentencesN:
            #print(translations_ambiguous_words[count])
            genders.append(set(get_german_determiners(translations_ambiguous_words[count])))
            male.append("male" in get_german_determiners(translations_ambiguous_words[count]))
            female.append("female" in get_german_determiners(translations_ambiguous_words[count]))
            print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(get_german_determiners(translations_ambiguous_words[count])), end='\n', file=fout)
            count += 1
            
    return (sum(1 for i in genders if ('male' in i and 'female' in i)), 
            male.count(True), female.count(True))

In [82]:
print(extract_articles(1, 335, 'hyp_original.txt', 'original_source-target_en-de_awesome-aligned.txt', 'tok.en_original.en', 'unique-words_translations_original_articles.txt'))
print(extract_articles(2, 335, 'hyp_disambiguated_male.txt', 'disambiguated_male_source-target_en-de_awesome-aligned.txt', 'tok.en_disambiguated_male.en', 'unique-words_translations_disambiguated_male_articles.txt'))
print(extract_articles(2, 335, 'hyp_disambiguated_female.txt', 'disambiguated_female_source-target_en-de_awesome-aligned.txt', 'tok.en_disambiguated_female.en', 'unique-words_translations_disambiguated_female_articles.txt'))

(307, 334, 308)
(307, 335, 307)
(278, 278, 335)


- Calculate gender in percentage for each sentence: percent of "male" vs. female in translations for each sentence

In [85]:
import re

# Extract articles of target tranlsated words
def extract_articles_percent(position, sentencesN, translationsIn, alignmentsIn, sourceIn, output):
    
    # Extract the position of the translated ambiguous word from each sentence
    positions_ambiguous_words = []
    for i in range(sentencesN):
        positions_ambiguous_words.append(position)
    
    # List with original translations
    translations_original = []
    with open(translationsIn, 'r') as fin:
        for line in fin:
            translations_original.append(line.strip())


    lineNumber = 0
    counter = 0
    indices = [] # a list of lists of indices of translated words for each ambiguous word
    with open(alignmentsIn, 'r') as alignments:
        for line in alignments:
            if (lineNumber == 100):
                lineNumber = 0
                counter += 1
            position = positions_ambiguous_words[counter] # exact position of ambiguous word
            regex = r"" + str(position) + r"-(\d)"
            if re.findall(regex, line): 
                indices.append([int(index) for index in re.findall(regex, line)])
            else:
                indices.append([999])
            lineNumber += 1

    #print(len(indices))
    #print(indices)

    lineNumber = 0
    translations_ambiguous_words = [] # a list of set of translations to each ambiguous word in source
    translated_ambiguous_words = [] 
    for translation in translations_original:
        tokens = translation.split(' ')
        if 999 not in indices[lineNumber]:
            for ind in indices[lineNumber]:
                translated_ambiguous_words.append(tokens[0]) # extract articles; currently assume index 0 for article position, TODO
        lineNumber += 1
        if (lineNumber % 100 == 0):
                translations_ambiguous_words.append(translated_ambiguous_words)
                translated_ambiguous_words = []


    #print(len(translations_ambiguous_words))
    
    # Add results to file

    # List with original source sentences
    source = []
    ambiguous_words = []
    with open(sourceIn, 'r') as fin:
        for line in fin:
            source.append(line.strip())
            tokens = line.split(' ')
            ambiguous_words.append(tokens[position])

    count = 0  
    genders = []
    male = []
    female = []
    with open(output, 'w') as fout:
        while count < sentencesN:
            #print(translations_ambiguous_words[count])
            genders.append(get_german_determiners(translations_ambiguous_words[count]))
            male.append("male" in get_german_determiners(translations_ambiguous_words[count]))
            female.append("female" in get_german_determiners(translations_ambiguous_words[count]))
            print(source[count] + ' | ' + ambiguous_words[count] + ' | ' + str(get_german_determiners(translations_ambiguous_words[count])), end='\n', file=fout)
            count += 1
     
    #print(genders)
    return (sum([i.count('male')/100 for i in genders])/sentencesN*100, 
            sum([i.count('female')/100 for i in genders])/sentencesN*100)

In [86]:
print(extract_articles_percent(1, 335, 'hyp_original.txt', 'original_source-target_en-de_awesome-aligned.txt', 'tok.en_original.en', 'unique-words_translations_original_articles.txt'))
print(extract_articles_percent(2, 335, 'hyp_disambiguated_male.txt', 'disambiguated_male_source-target_en-de_awesome-aligned.txt', 'tok.en_disambiguated_male.en', 'unique-words_translations_disambiguated_male_articles.txt'))
print(extract_articles_percent(2, 335, 'hyp_disambiguated_female.txt', 'disambiguated_female_source-target_en-de_awesome-aligned.txt', 'tok.en_disambiguated_female.en', 'unique-words_translations_disambiguated_female_articles.txt'))

(78.65970149253735, 14.880597014925373)
(81.96119402985079, 14.241791044776114)
(8.271641791044772, 86.90149253731335)
